In [1]:
from custom_dataset import CustomDataset
import matplotlib.pyplot as plt
import argparse
import os
import time 
import numpy as np

import torch
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torchvision.utils import save_image
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.transforms.functional as TF
from torchvision.utils import save_image

from model_8_hdim_edited import Autoencoder

2023-08-17 09:41:35.976975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 09:41:37.069158: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
""" This script is an example of VAE training in PyTorch. The code was adapted from:
https://github.com/pytorch/examples/blob/master/vae/main.py """

## Arguments
args = argparse.Namespace(
    batch_size=32768,
    input_size=16,
    epochs=50,
    no_cuda=False,
    log_interval=500,
    model='mse_vae',
    log_dir='mse_vae'
)


## Cuda
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda:3" if args.cuda else "cpu")

# ## Data Parallelism
# if args.cuda and torch.cuda.device_count() > 1:
#     print("Using", torch.cuda.device_count(), "GPUs for Data Parallelism!")
#     model = nn.DataParallel(Autoencoder(args.input_size)).to(device)
# # else:
model = Autoencoder(args.input_size).to(device)

kwargs = {'num_workers': 48, 'pin_memory': True} if args.cuda else {}

transform = transforms.Compose([
    transforms.Resize((args.input_size, args.input_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def random_mask(img_data, device):
    # Generate a binary mask with a random percentage of values set to 1
    mask_percentage = np.random.uniform(0.05, 0.2)
    mask = torch.rand(img_data.shape, device=device) < mask_percentage
    # Apply the mask to the random_image to replace values with -1000
    dummy_value = -1
    masked_image = torch.where(mask, dummy_value, img_data)
    return masked_image

NameError: name 'patch_size' is not defined

In [ ]:
# ## ---------------------------------------------------
train_dataset = CustomDataset(data_path='/mayo_atlas/home/m296984/MAIN_CHAIN_LIVER_RESULTS/train_16x16_patches', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)
test_dataset = CustomDataset(data_path='/mayo_atlas/home/m296984/MAIN_CHAIN_LIVER_RESULTS/test_16x16_patches', transform=transform)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)


In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Run VAE
        recon_batch, mu, logvar = model(data)
        # Compute loss
        rec, kl = model.module.loss_function(recon_batch, data, mu, logvar)
        
        total_loss = rec + kl * 0.1
        # total_loss = rec + kl 
        total_loss.backward()
        train_loss += total_loss.item()
        
        optimizer.step()
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tMSE: {:.6f}\tKL: {:.6f}\tlog_sigma: {:.6f}\tLR: {:.13f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                rec.item() / len(data),
                kl.item() / len(data),
                model.module.log_sigma if isinstance(model, nn.DataParallel) else model.log_sigma,
                scheduler.get_last_lr()[0]))
                
    train_loss /=  len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss))

In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(tqdm(test_loader)):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            rec, kl = model.module.loss_function(recon_batch, data, mu, logvar)
            test_loss += rec + kl * 0.1
            # test_loss += rec + kl
            if i == 2:
                n = min(data.size(0), 20)
                comparison = torch.cat([data[:n], recon_batch.view(args.batch_size_test, -1, args.input_size, args.input_size)[:n]])
                save_image(comparison.cpu(), '/mayo_atlas/home/m296984/MAIN_CHAIN_LIVER_RESULTS/SAVED TEST_IMAGES/{}/reconstruction_{}.png'.format(args.log_dir, str(epoch)), nrow=n)
                
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))